In [1]:
import re
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
# let's take a look Kareem Abdul-Jabbar

url = 'https://www.basketball-reference.com/players/a/abdulka01.html'

df = pd.read_html(url)

In [3]:
df_kaj = df[0].loc[:, :'PTS']
df_kaj.tail()

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
16,1985-86,38.0,LAL,NBA,C,79,79.0,33.3,9.6,16.9,...,0.765,1.7,4.4,6.1,3.5,0.8,1.6,2.6,3.1,23.4
17,1986-87,39.0,LAL,NBA,C,78,78.0,31.3,7.2,12.7,...,0.714,1.9,4.8,6.7,2.6,0.6,1.2,2.4,3.1,17.5
18,1987-88,40.0,LAL,NBA,C,80,80.0,28.9,6.0,11.3,...,0.762,1.5,4.5,6.0,1.7,0.6,1.2,2.0,2.7,14.6
19,1988-89,41.0,LAL,NBA,C,74,74.0,22.9,4.2,8.9,...,0.739,1.4,3.1,4.5,1.0,0.5,1.1,1.3,2.6,10.1
20,Career,NaN,NaN,NBA,NaN,1560,625.0,36.8,10.2,18.1,...,0.721,2.4,7.6,11.2,3.6,0.9,2.6,2.7,3.0,24.6


What question to answer?

- Stats for teams starting lineup in previous year --> number of wins in regular season?

Let's start with the spurs in 2006:

In [4]:
# looks like top 5 rank can be a decent standin for starting lineup.

url = 'https://www.basketball-reference.com/teams/SAS/2006.html'

response = requests.get(url)
response.status_code

200

In [5]:
soup = BeautifulSoup(response.text, 'lxml')

# <!-- is open comment

# --> is close comment

We have discovered that the tables are commented out. The theory is that this is done in order to make the tables sortable. There's perhaps a javascript applet that applies here.

In [6]:
# uncomment the html
macaroni = soup.prettify().split('<!--')
macaroni = [mac.strip('-->') for mac in macaroni]

macaroni[20] # example table

'\n   <div class="table_outer_container">\n      <div class="overthrow table_container" id="div_team_and_opponent">\n  <table class="suppress_all stats_table" id="team_and_opponent" data-cols-to-freeze=1><caption>Team and Opponent Stats Table</caption>\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>      \n      <tr>\n         <th aria-label="&nbsp;" data-stat="player" scope="col" class=" poptip sort_default_asc center" >&nbsp;</th>\n         <th aria-label="Games" data-stat="g" scope="col" class=" poptip sort_default_asc center" data-tip="Games" >G</th>\n         <th aria-label="Minutes Played" data-stat="mp" scope="col" class=" poptip center" data-tip="Minutes Played" >MP</th>\n         <th aria-label="Field Goals" data-stat="fg" scope="col" class=" poptip center" data-tip="Field Goals" >FG</th>\n         <th aria-label="Field Goal Attempts" data-stat="fga" scope="col" class=

In [93]:
tables = []

for nood in macaroni:
    udon = BeautifulSoup(nood, 'lxml').find_all('table')
    if udon != []:
        tables.append(udon[0])
    
for table in tables:
    print(table.text)# looks good!



         Roster Table
        














           No.
          

           Player
          

           Pos
          

           Ht
          

           Wt
          

           Birth Date
          



           Exp
          

           College
          





           17
          


            Brent Barry
           


           SG
          

           6-6
          

           185
          

           December 31, 1971
          


            us
           


           10
          


            Oregon State University
           




           12
          


            Bruce Bowen
           


           SF
          

           6-7
          

           185
          

           June 14, 1971
          


            us
           


           9
          


            California State University, Fullerton
           




           21
          


            Tim Duncan
           


           PF
          

           6-11
          

 

In [83]:
# expirmenting with random table
sample_table = tables[3]

rows = sample_table.find_all('tr')

data = []

for row in rows:
    row_data = []
    
    for entry in row.find_all('th'):
        row_data.append(entry.text)
    
    
    for entry in row.find_all('td'):
        row_data.append(entry.text)
    data.append(row_data)

df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,,Advanced,Offense Four Factors,Defense Four Factors,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,...,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attendance
2,Team,63,19,61,21,6.82,-0.13,6.69,107.3,99.6,...,.513,13.6,26.0,.209,.457,13.5,74.2,.220,SBC Center,"770,677"
3,Lg Rank,2,29,1,1,1,23,1,10,1,...,3,13,20,29,1,19,8,4,,8


In [85]:
# get all tables into a list of dfs

#########################
#       template        #
#########################

# sample_table = tables[3]

# rows = sample_table.find_all('tr')

# data = []
# count = 0

# for row in rows:
#     row_data = []
    
#     for entry in row.find_all('th'):
#         row_data.append(entry.text)
    
    
#     for entry in row.find_all('td'):
#         row_data.append(entry.text)
#     data.append(row_data)

# df = pd.DataFrame(data)
# df
###############################

dfs = []

for table in tables:
    rows = table.find_all('tr')
    data = []
    
    for row in rows:
        row_data = []
        
        for entry in row.find_all('th'): # headers
            row_data.append(entry.text)
        
        for entry in row.find_all('td'): # datums
            row_data.append(entry.text)
            
        data.append(row_data)
        
    dfs.append(pd.DataFrame(data))

In [107]:
dfs[10]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,Totals,Position Estimate,+/- Per 100 Poss.,Turnovers,Fouls Committed,Fouls Drawn,Misc.,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Rk,,Age,G,MP,PG%,SG%,SF%,PF%,C%,...,On-Off,BadPass,LostBall,Shoot,Off.,Shoot,Off.,PGA,And1,Blkd
2,1,Tim Duncan,29,80,2784,,,,71%,29%,...,+4.2,59,72,118,31,244,10,571,61,65
3,2,Bruce Bowen,34,82,2755,,,88%,12%,,...,+0.7,30,24,68,8,30,40,285,6,22
4,3,Tony Parker,23,80,2715,92%,8%,,,,...,+7.3,111,68,41,32,154,9,1065,48,78
5,4,Michael Finley,32,77,2038,,43%,53%,4%,,...,-4.4,31,19,57,6,21,5,262,3,12
6,5,Manu Ginobili,28,65,1813,,83%,17%,,,...,+7.3,60,34,59,15,120,48,536,23,33
7,6,Rasho Nesterovic,29,80,1515,,,,,100%,...,+2.9,22,12,124,9,13,12,76,2,13
8,7,Nazr Mohammed,28,80,1389,,,,12%,88%,...,-4.0,18,30,129,18,72,6,87,19,55
9,8,Brent Barry,34,74,1259,3%,91%,6%,,,...,+3.6,31,14,47,5,22,7,278,5,12
